In [1]:
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('data.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)

df2 = pd.read_csv('data.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)


C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df1.label[df1.label == 1]="acpt"
df1.label[df1.label == 0]="unac"

df2.label[df2.label == 1]="acpt"
df2.label[df2.label == 0]="unac"

C:\Users\moshi\AppData\Local\Temp/ipykernel_8264/3898086040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.label[df1.label == 1]="acpt"
C:\Users\moshi\AppData\Local\Temp/ipykernel_8264/3898086040.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.label[df2.label == 1]="acpt"


In [4]:
# taking acceptable and deleting unacceptable
df3 = df1[~df1.label.str.contains('acpt')]

In [5]:
# randomly picking 250 rows
df3 = df3.sample(n=250, replace=True)

In [6]:
df3["label"].value_counts()

unac    250
Name: label, dtype: int64

In [7]:
# taking unacceptable and deleting unacceptable
df4 = df2[~df2.label.str.contains("unac")]

In [8]:
# randomly picking 250 rows
df4 = df4.sample(n=250, replace=True)

In [9]:
df4["label"].value_counts()

acpt    250
Name: label, dtype: int64

In [10]:
#concating acceptable and unacceptable
frames = [df3, df4]

temp = pd.concat(frames)

In [11]:
temp["label"].value_counts()

unac    250
acpt    250
Name: label, dtype: int64

In [12]:
temp.head()

,label,text
2551,unac,Cheryl stood the shelf with books.
793,unac,Mary considered to have loved her.
3733,unac,"The book, including all the chapters in the fi..."
4821,unac,I don't know that to agree with him or not.
5619,unac,Himself sang to George.


In [13]:
temp.tail()

,label,text
6909,acpt,Jim was happily chopping logs.
3901,acpt,Kim and Sandy met in Seoul in the lobby of the...
8319,acpt,The owl hated the evil bat and the wise eagle
4739,acpt,Who is clever?
4022,acpt,These are similar to the bottles.


In [14]:
temp["label"].value_counts()

unac    250
acpt    250
Name: label, dtype: int64

In [15]:
text = temp['text'].values.tolist()
emotion= temp['label'].tolist()

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [17]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [18]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 16
	99percentile : 20


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 7
	95percentile : 12
	99percentile : 18


In [19]:
model = t.get_classifier()

Downloading:   0%|          | 0.00/347M [00:00<?, ?B/s]

In [20]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [21]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
67/67 [==============================] - 684s 10s/step - loss: 0.6956 - accuracy: 0.4300 - val_loss: 0.6944 - val_accuracy: 0.4800
Epoch 2/5
67/67 [==============================] - 849s 13s/step - loss: 0.6815 - accuracy: 0.5650 - val_loss: 0.6649 - val_accuracy: 0.5300
Epoch 3/5
67/67 [==============================] - 851s 13s/step - loss: 0.5532 - accuracy: 0.7550 - val_loss: 0.6469 - val_accuracy: 0.6600
Epoch 4/5
67/67 [==============================] - 853s 13s/step - loss: 0.2078 - accuracy: 0.9450 - val_loss: 0.9025 - val_accuracy: 0.6100
Epoch 5/5
67/67 [==============================] - 679s 10s/step - loss: 0.0877 - accuracy: 0.9825 - val_loss: 0.8638 - val_accuracy: 0.6500


In [22]:
class_names=["acpt","unac"]

In [23]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

        acpt       0.66      0.72      0.69        54
        unac       0.63      0.57      0.60        46

    accuracy                           0.65       100
   macro avg       0.65      0.64      0.64       100
weighted avg       0.65      0.65      0.65       100



array([[39, 15],
       [20, 26]], dtype=int64)